In [1]:
from web3 import Web3
from solcx import install_solc, compile_standard
import solcx
import json
import os

install_solc(version='0.8.0')

onchain = Web3(Web3.HTTPProvider('http://localhost:8545')) #on-chain

s1 = Web3(Web3.HTTPProvider('http://localhost:8546')) #shard 1
s2 = Web3(Web3.HTTPProvider('http://localhost:8547')) #shard 2
s3 = Web3(Web3.HTTPProvider('http://localhost:8548')) #shard 3

onchain.isConnected(), s1.isConnected(), s2.isConnected(), s3.isConnected()

(True, True, True, True)

In [2]:
# codice per far inserire all'utente il path dello smart contract da deployare

print("Inserisci la path del file solidity dello smart contract di cui fare il deploy")
file_path = input()

# controllo che il file esista effettivamente e che sia di tipo .sol
while not os.path.exists(file_path) and os.path.splitext(file_path)[1] != ".sol":
    os.system('clear')
    print("La path deve esistere e il file deve essere di tipo .sol")
    file_path = input()
    
# Leggo il file contenente lo smart contract
with open(file_path, "r") as file:
    prova_on_chain_file = file.read()

Inserisci la path del file solidity dello smart contract di cui fare il deploy


In [3]:
try:
    # Compilo il file contenente lo smart contract
    compiled_sol = compile_standard(
        {
            "language": "Solidity",
            "sources": {"prova_on_chain.sol": {"content": prova_on_chain_file}},
            "settings": {
                "outputSelection": {
                    "*": {
                        "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"] # output needed to interact with and deploy contract 
                    }
                }
            },
        },
        solc_version="0.8.0",
    )

    # get bytecode
    bytecode = compiled_sol["contracts"]["prova_on_chain.sol"]["on_chain_manager"]["evm"]["bytecode"]["object"]
    # get abi
    abi = json.loads(compiled_sol["contracts"]["prova_on_chain.sol"]["on_chain_manager"]["metadata"])["output"]["abi"]

except Exception as exception: 
    os.system('clear')
    #viene preso l'output restituito da solc e stampata sulla console solo l'informazione di interesse. 
    jsonError = json.loads(json.dumps(exception.__dict__))
    items = jsonError["error_dict"][0]
    print("Attenzione! E' stato generato il seguente errore: " + "\"" + items["message"] + "\", \n"
          + "In "+ "\"" +items["sourceLocation"]["file"]+ "\""+
          ", con codide di errore: " + items["errorCode"]+ ".")